In [1]:
import numpy as np
import cv2
from cv2 import aruco, cvtColor, COLOR_BGR2GRAY, LUT
import matplotlib.pyplot as plt
from glob import glob
import pandas as pd
from funktionen import *
from scipy.sparse import lil_matrix
from scipy.optimize import least_squares
from scipy import stats

In [38]:
pictures = pd.DataFrame([[i, 0, pfad, None, None, None, None, None, None] for i, pfad in enumerate(glob("../../serie2/*.jpg"))], columns=["img", "kamera",
                        "pfad", "t1", "t2", "t3", "r1", "r2", "r3"]).astype({"kamera": int, "pfad": str, "img": int, "t1": float, "t2": float, "t3": float, "r1": float, "r2": float, "r3": float}).set_index(["img"])
# np.c_[t.T,rod.T]
pictures.dtypes

kamera      int64
pfad       object
t1        float64
t2        float64
t3        float64
r1        float64
r2        float64
r3        float64
dtype: object

In [10]:
aruco_dict = aruco.extendDictionary(32, 3)
parameter = aruco.DetectorParameters()
parameter.cornerRefinementMethod = aruco.CORNER_REFINE_SUBPIX
LUT_IN = [0, 158, 216, 255]
LUT_OUT = [0, 22, 80, 176]
lut = np.interp(np.arange(0, 256),
                LUT_IN, LUT_OUT).astype(np.uint8)

In [11]:
corners = []
for j, img in pictures.iterrows():
    imgCV = cv2.imread(img['pfad'])
    gray = cvtColor(imgCV, COLOR_BGR2GRAY)
    tmp_corners, tmp_ids, t = aruco.detectMarkers(
        gray, aruco_dict, parameters=parameter)
    for c, i in zip(tmp_corners, tmp_ids):
        size = np.linalg.norm(c[0][0] - c[0][1])
        # print(size)
        if size < 100:
            continue
        for k in range(len(c[0])):
            corners.append([j, i[0], k, c[0][k][0], c[0][k][1]])
corners = pd.DataFrame(corners, columns=["img", "marker", "ecke", "x", "y"])
corners.set_index(["img", "marker", "ecke"], inplace=True)


In [12]:
corners

x            y
img marker ecke                          
0   0      0     2161.458008  1439.255981
           1     2893.765137  1405.369263
           2     3012.325195  2103.274414
           3     2172.485352  2142.125732
    13     0      201.722046  2107.381348
...                      ...          ...
23  16     3     4270.064941   692.533813
    10     0     2649.208008   843.520996
           1     2481.939209   847.895935
           2     2478.655762   678.135925
           3     2646.150146   673.733337

[524 rows x 2 columns]

In [ ]:
#corners.drop(index=(3,0,1), inplace=True)

In [13]:
ca = corners.reset_index().set_index(["marker", "ecke"])
matched = ca.join(ca, lsuffix="_a", rsuffix="_b").query(
    'img_a < img_b').query("marker < 17")
matched.reset_index(inplace=True)
ca

img            x            y
marker ecke                               
0      0       0  2161.458008  1439.255981
       1       0  2893.765137  1405.369263
       2       0  3012.325195  2103.274414
       3       0  2172.485352  2142.125732
13     0       0   201.722046  2107.381348
...          ...          ...          ...
16     3      23  4270.064941   692.533813
10     0      23  2649.208008   843.520996
       1      23  2481.939209   847.895935
       2      23  2478.655762   678.135925
       3      23  2646.150146   673.733337

[524 rows x 3 columns]

In [37]:
coords = pd.DataFrame([[0,0,-7.8,7.8,0.0],
                       [0, 1, 7.8, 7.8, 0.0],
                       [0, 2, 7.8, -7.8, 0.0],
                       [0, 3, -7.8, -7.8, 0.0]], columns=["marker", "ecke", "x", "y", "z"]).astype({"marker": int, "ecke": int}).set_index(["marker", "ecke"])
coords

x    y    z
marker ecke               
0      0    -7.8  7.8  0.0
       1     7.8  7.8  0.0
       2     7.8 -7.8  0.0
       3    -7.8 -7.8  0.0

In [35]:
mtx = np.array([[3.48172482e+03, 0.00000000e+00, 2.30250916e+03],
                [0.00000000e+00, 3.47313834e+03, 1.31660435e+03],
                [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])
mtx = np.array([[3440., 0., 2307],
                [0., 3440., 1298.],
                [0.,    0.,    1.]], dtype=np.float64)
dist = np.array([-1.41372220e-02,  2.43909839e-01,
                1.15659062e-03,  1.28627153e-04,  -5.96950573e-01], dtype=np.float64)
dist = np.array([0,0,0,0,0], dtype=np.float64)
kameras = pd.DataFrame([[i, mtx[0, 0], mtx[1, 1], mtx[0, 2], mtx[1, 2],
                        dist[0], dist[1], dist[2], dist[3], dist[4]] for i in pictures["kamera"].unique()],
                       columns=["kamera", "fx", "fy", "cx", "cy", "d1", "d2", "d3", "d4", "d5"]).astype({"kamera": int}).set_index("kamera")
kameras

,fx,fy,cx,cy,d1,d2,d3,d4,d5
kamera,,,,,,,,,
0,3440.0,3440.0,2307.0,1298.0,0.0,0.0,0.0,0.0,0.0


In [36]:
def get_kamera(kameras, kid):
    kamera = kameras.iloc[kid]
    mtx = np.array([[kamera["fx"], 0, kamera["cx"]],
                    [0, kamera["fy"], kamera["cy"]],
                    [0, 0, 1]])
    dist = np.array(kamera[["d1", "d2", "d3", "d4", "d5"]])
    return mtx, dist

In [17]:
def neues_bild(coords, pictures, kameras, corners):
    lp = corners.join(coords, lsuffix="_img", rsuffix="").query(
        "marker < 17").dropna().join(pictures, lsuffix="", rsuffix="_cam")
    lp = lp[lp["r1"].isnull()]
    bild = lp.groupby("img").count().sort_values(
        "x_img", ascending=False).first_valid_index()
    passpunkte = lp.query('img == ' + str(bild)).reset_index()
    pp = passpunkte[['x_img', 'y_img', 'x', 'y', 'z']].values
    kid = pictures.query('img == ' + str(bild))['kamera'].values[0 ]
    k, dist = get_kamera(kameras, kid)   
    
    if len(pp) >= 4:
        _, r, t = cv2.solvePnP(pp[:, 2:], pp[:, :2], k, dist)
        rod, _ = cv2.Rodrigues(r)
        p = pictures.query('img == ' + str(bild))
        p[["t1", "t2", "t3", "r1", "r2", "r3"]] = [t[0][0], t[1][0], t[2][0], r[0][0], r[1][0], r[2][0]]
        pictures.update(p)
    return pictures

In [58]:
pictures = neues_bild(coords, pictures, kameras, corners)
pictures#.sort_index()

IndexError: index 0 is out of bounds for axis 0 with size 0

In [79]:
def wolke_zeigen(coords, pictures):

    # pip install plotly

    import plotly.graph_objs as go


    coords_img = []
    for _, p in pictures.dropna().iterrows():
        rt = p[['r1', 'r2', 'r3']].to_numpy(dtype=np.float64)
        R = cv2.Rodrigues(rt)[0]
        R = np.linalg.inv(R)
        t = p[['t1', 't2', 't3']].to_numpy(dtype=np.float64).T
        t = -R@t
        coords_img.append(t)
    coords_img = np.array(coords_img)

    # Create a 3D scatter plot
    fig = go.Figure(data=[go.Scatter3d(x=coords['x'], y=coords['y'], z=coords['z'], mode='markers', name='Original Points'),
                          go.Scatter3d(x=coords_img[:, 0], y=coords_img[:, 1], z=coords_img[:, 2],
                                       mode='markers', name='Kameras')])

    # Add labels to the plot
    fig.update_layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'),
                      title='Rotation and Translation Vectors in 3D')
    fig.layout.scene.camera.projection.type = "orthographic"
    fig.show()


wolke_zeigen(coords, pictures)

In [20]:
def neue_koordinaten(coords, pictures, kameras):
    pic_a = pictures.reset_index().rename(
        columns={"img": "img_a"}).set_index("img_a")
    pic_b = pictures.reset_index().rename(
        columns={"img": "img_b"}).set_index("img_b")
    matched = ca.add_suffix('_a').join(ca.add_suffix('_b')).query(
        'img_a < img_b').query("marker < 17").join(coords.add_suffix("_ca"))
    #print(matched)
    matched = matched[matched["x_ca"].isnull()].reset_index().set_index([
        "img_a", "img_b"])
    matched = matched.join(pic_a.add_suffix("_a"))
    matched = matched.join(pic_b.add_suffix("_b"))
    #print(matched)
    matched.dropna(subset=['t1_a', 't1_b'], inplace=True)
    matched = matched.reset_index().set_index(["marker", "ecke"])

    beste = matched.groupby(["marker"]).count().sort_values(
        "img_a", ascending=False)

    #print(beste)

    for bester in beste.reset_index().values[:, 0]:
        new = []
        # print(bester)
        for i in range(4):
            cneu = []

            for i, v in matched.query("marker =="+str(bester)).query("ecke =="+str(i)).iterrows():
                #print(v)
                K1, dist1 = get_kamera(kameras, v['kamera_a'])
                K2, dist2 = get_kamera(kameras, v['kamera_b'])
                R1 = cv2.Rodrigues(
                    np.array(list(v[['r1_a', 'r2_a', 'r3_a']].to_numpy(dtype=np.float64))))[0]
                R2 = cv2.Rodrigues(
                    np.array(list(v[['r1_b', 'r2_b', 'r3_b']].to_numpy(dtype=np.float64))))[0]
                t1 = v[['t1_a', 't2_a', 't3_a']].to_numpy(dtype=np.float64)
                t2 = v[['t1_b', 't2_b', 't3_b']].to_numpy(dtype=np.float64)
                P1 = np.c_[R1, t1]
                P2 = np.c_[R2, t2]
                pt1 = np.array(list(v[['x_a', 'y_a']]))
                pt2 = np.array(list(v[['x_b', 'y_b']]))
                p1n = cart2hom(cv2.undistortPoints(pt1, K1, dist1)[0][0])
                p2n = cart2hom(cv2.undistortPoints(pt2, K2, dist2)[0][0])

                xy = reconstruct_one_point(p1n, p2n, P1, P2)
                xy /= xy[3]
                cneu.append(xy[0:3])
            
            if len(cneu) == 0:
                continue
            data = pd.DataFrame(cneu, columns=["x", "y", "z"])
            # print(data['x'].std())
            if data['x'].std() > 0.2 or data['y'].std() > 0.2 or data['z'].std() > 0.2:
                # print(data)
                continue
            # print(data[(np.abs(stats.zscore(data)) < 2).all(axis=1)])
            median = data.mean().values
            new.append([bester, i[1], median[0], median[1], median[2]])
        # print(new)
        if len(new) == 4:
            new = pd.DataFrame(new, columns=["marker", "ecke", "x", "y", "z"]).set_index(
                ["marker", "ecke"])
            coords = pd.concat([coords, new])
            return coords
    return coords

In [78]:
pictures = neues_bild(coords, pictures, kameras, corners)

/tmp/ipykernel_98841/4252915202.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [75]:
coords = neue_koordinaten(coords, pictures, kameras)

In [ ]:
for i in range(24):
    pictures = neues_bild(coords, pictures, kameras, corners)
    coords = neue_koordinaten(coords, pictures, kameras)

In [34]:
wolke_zeigen(coords, pictures)

### Ausrichtung

In [ ]:
def ausrichten(coords, pictures):
    bodenpunkte = coords.query("marker == 0 or marker == 6 or marker == 15 or marker == 3 or marker == 11 or marker == 13 or marker == 14 or marker == 4").query(
        "ecke == 1 or ecke == 2")
    hoeherepunkte = coords.query("marker == 0 or marker == 6 or marker == 15 or marker == 3 or marker == 11 or marker == 13 or marker == 14 or marker == 4").query(
        "ecke == 0 or ecke == 3")

    t_boden = bodenpunkte.mean().values
    vec2 = hoeherepunkte.mean() - t_boden
    laenge = np.linalg.norm(vec2)
    vec2 /= laenge
    b = vec2.values
    faktor = 3.4 / laenge

    a = np.array([0, 0, 1])

    v = np.cross(a, b)
    c = np.dot(a, b)
    s = np.linalg.norm(v)
    kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    rotation_matrix = np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))

    coords[['x', 'y', 'z']] -= t_boden
    coords[['x', 'y', 'z']] *= faktor
    coords[['x', 'y', 'z']] = coords[['x', 'y', 'z']].dot(rotation_matrix)

    for pict in pictures.values:
        t = pict[0:3].T
        R_org = cv2.Rodrigues(pict[3:])[0]
        R = np.linalg.inv(R_org)
        t = -R@t
        R_neu = np.linalg.inv(rotation_matrix@R_org)
        pict[0:3] = -R_neu@(((t-t_boden)*faktor)@rotation_matrix).T
        pict[3:] = cv2.Rodrigues(R_neu)[0].T
    return coords, pictures

In [ ]:
def vergleich(imgs, coords, pictures, kameras):
    d = []
    for nr in pictures.index:
        r = pictures.query("img == "+str(nr))[["r1", "r2", "r3"]].values
        t = pictures.query("img == "+str(nr))[["x", "y", "z"]].values
        kp = kameras.query("kamera == 1").values
        mtx = np.array([[kp[0][0], 0, kp[0][2]], [
            0, kp[0][1], kp[0][3]], [0, 0, 1]])
        dist = kp[0][4:]
        img_points, _ = cv2.projectPoints(coords.values, r, t, mtx, dist)
        img_points = pd.DataFrame(
            img_points.reshape(-1, 2), columns=["x_neu", "y_neu"])
        img_points['img'] = nr
        img_points[["marker", "ecke"]] = coords.reset_index()[
            ["marker", "ecke"]]
        img_points.set_index(["marker", "ecke", "img"], inplace=True)
        j = img_points.join(corners, lsuffix="_neu", rsuffix="_alt").dropna()
        j['dx'] = j['x_neu'] - j['x']
        j['dy'] = j['y_neu'] - j['y']
        j['d'] = np.sqrt(j['dx']**2 + j['dy']**2)

        if (j[['d']].std().values[0] > 40):
            print("Entferne Bild ", nr+1)
            pictures.drop(nr, inplace=True)
        elif j[j['d'] > 15].count().values[0] > 1:
            for i in j[j['d'] > 15].index:
                print("Entferne Passpunkt ", i)
                print(i)
                coords.drop(i[:2], inplace=True)


# vergleich(imgs, coords, pictures, kameras)

### Ausgleichung

In [ ]:
def passpunkt_groesse(coords):
    l = []
    x0 = []
    A = []
    m = 3.4
    for marker in coords.reset_index()['marker'].unique():
        if marker == 0:
            m = 15.6
        else:
            m = 3.4
        m1 = np.where(coords.index == (marker, 0))[0][0]
        m2 = np.where(coords.index == (marker, 1))[0][0]
        m3 = np.where(coords.index == (marker, 2))[0][0]
        m4 = np.where(coords.index == (marker, 3))[0][0]
        l.append([m, m*2**0.5, m, m, m*2**0.5, m])
        A.append([m1*2, m1*2 + 1, m2*2, m2*2+1])
        A.append([m1*2, m1*2 + 1, m3*2, m3*2+1])
        A.append([m1*2, m1*2 + 1, m4*2, m4*2+1])
        A.append([m2*2, m2*2 + 1, m3*2, m3*2+1])
        A.append([m2*2, m2*2 + 1, m4*2, m4*2+1])
        A.append([m3*2, m3*2 + 1, m4*2, m4*2+1])
    l = np.array(l)
    A = np.array(A)
    return A, l.ravel()

In [ ]:
pictures

In [ ]:
def ausgleichung(coords, pictures, kameras, corners):
    lp = corners.join(coords, lsuffix="_img", rsuffix="").query(
        "marker < 17").join(pictures, lsuffix="", rsuffix="_cam").dropna()

    Apg, lpg = passpunkt_groesse(coords)

    kamera = kameras.reset_index().values
    kamera_ids = np.array(kamera[:, 0], dtype=np.int32)

    bilder = pictures.reset_index().dropna().drop(
        'pfad', axis=1).to_numpy(dtype=np.float32)
    bilder_ids = np.array(bilder[:, :2], dtype=np.int32)

    #print(bilder_ids)

    passpunkte = coords.reset_index().values
    passpunkte_ids = np.array(passpunkte[:, :2], dtype=np.int32)

    #print(lp)
    messung = lp.reset_index().to_numpy()[:, :5]
    messung_ids = np.array(messung[:, :3], dtype=np.int32)

    #print(kamera_ids)
    #print(lpg)

    l = np.hstack(
        (messung[:, -2:].astype(np.float32).ravel(), lpg.ravel()), dtype=np.float32)

    x0 = np.hstack((kamera[:, 1:].ravel(),
                    bilder[:, 2:].ravel(),
                    passpunkte[:, 2:].ravel()),dtype=np.float32)

    A = lil_matrix((len(l), len(x0)), dtype=int)

    n_camera = len(kamera)
    n_bilder = len(bilder)
    n_passpunkte = len(passpunkte)
    n_messungen = len(messung)
    messung_bild_id = np.empty(n_messungen, dtype=np.int32)
    messung_kamera_id = np.empty(n_messungen, dtype=np.int32)
    messung_passpunkt_id = np.empty(n_messungen, dtype=np.int32)
    num_cam_param = 9
    num_bild_param = 6
    num_pass_param = 3

    offset = n_camera*num_cam_param + n_bilder * num_bild_param

    for i, a in enumerate(Apg):
        zeile = n_messungen + i
        for j in a:
            A[zeile, offset+j] = 1

    for i, m in enumerate(messung):
        #print(m)
        bild_id, = np.where(bilder_ids[:, 0] == m[2])
        bild_id = bild_id[0]
        messung_bild_id[i] = bild_id

        camera_id_array,  = np.where(kamera_ids[:] == bilder[bild_id, 1])
        camera_id = camera_id_array[0]
        messung_kamera_id[i] = camera_id

        passpunkt_id_array, = np.where(
            (passpunkte_ids[:, 0] == m[0]) & (passpunkte_ids[:, 1] == m[1]))
        passpunkt_id = passpunkt_id_array[0]
        messung_passpunkt_id[i] = passpunkt_id

        offset = camera_id*num_cam_param
        A[2*i:2*i+2, offset:offset+num_cam_param] = 1

        offset = n_camera * num_cam_param + bild_id * num_bild_param
        A[2*i:2*i+2, offset:offset + num_bild_param] = 1

        offset = n_camera*num_cam_param + n_bilder * \
            num_bild_param + passpunkt_id * num_pass_param
        A[2*i:2*i+2, offset:offset + num_pass_param] = 1

    def project(x0: NDArray[np.float32]) -> NDArray[np.float32]:
        p = np.empty(len(l), dtype=np.float32)

        K = []
        dist = []
        for i in range(n_camera):
            offset = i * num_cam_param
            fx = x0[offset]
            fy = x0[offset + 1]
            cx = x0[offset + 2]
            cy = x0[offset + 3]
            K.append(np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]]))
            dist.append(np.array([x0[offset + 4], x0[offset + 5],
                                  x0[offset + 6], x0[offset + 7], x0[offset + 8]]))

        r = []
        t = []
        for i in range(n_bilder):
            offset = n_camera * num_cam_param + \
                i*num_bild_param

            t.append(x0[offset:offset+3])
            r.append(x0[offset+3:offset+6])

        X = []
        for i in range(n_passpunkte):
            offset = n_camera * num_cam_param + \
                n_bilder*num_bild_param + \
                num_pass_param * i
            X.append(x0[offset:offset+3].reshape(1, 3))

        for i, m in enumerate(messung):
            punkt, _ = cv2.projectPoints(
                X[messung_passpunkt_id[i]], r[messung_bild_id[i]], t[messung_bild_id[i]], K[messung_kamera_id[i]], dist[messung_kamera_id[i]])
            # print(i*2+1)
            p[i*2] = punkt[0, 0, 0]
            p[i*2+1] = punkt[0, 0, 1]

        for i, marker in enumerate(coords.reset_index()['marker'].unique()):
            if marker == 0:
                m = 15.6
            else:
                m = 3.4
            m1 = np.where(coords.index == (marker, 0))[0][0]
            m2 = np.where(coords.index == (marker, 1))[0][0]
            m3 = np.where(coords.index == (marker, 2))[0][0]
            m4 = np.where(coords.index == (marker, 3))[0][0]
            p[i+0] = np.linalg.norm(coords.values[m1] - coords.values[m2])*100
            p[i+1] = np.linalg.norm(coords.values[m1] - coords.values[m3])*100
            p[i+2] = np.linalg.norm(coords.values[m1] - coords.values[m4])*100
            p[i+3] = np.linalg.norm(coords.values[m2] - coords.values[m3])*100
            p[i+4] = np.linalg.norm(coords.values[m2] - coords.values[m4])*100
            p[i+5] = np.linalg.norm(coords.values[m3] - coords.values[m4])*100

        return np.array(p.ravel()-l, dtype=np.float32)

    #x0 = np.array(x0, dtype=float)
    res = least_squares(project, x0, jac_sparsity=A, verbose=2,
                        x_scale='jac', method='trf', ftol=1e-5)  # type: ignore

    x = res.x

    kameras = pd.DataFrame(np.c_[kamera_ids, x[:num_cam_param * n_camera].reshape(n_camera, num_cam_param)],
                           columns=["kamera", "fx", "fy", "cx", "cy", "d1", "d2", "d3", "d4", "d5"]).astype({"kamera": int}).set_index("kamera")

    offset = num_cam_param * n_camera
    pictures = pictures.update(pd.DataFrame(np.c_[bilder_ids[:, 1], x[offset:offset+num_bild_param*n_bilder].reshape(n_bilder, num_bild_param)], columns=["img", "x", "y", "z", "r1", "r2", "r3"]).astype({"img": int}).set_index("img"))

    offset = num_cam_param * n_camera + num_bild_param*n_bilder
    coords = pd.DataFrame(np.c_[passpunkte_ids, x[offset:offset+num_pass_param*n_passpunkte].reshape(n_passpunkte, num_pass_param)], columns=["marker", "ecke", "x", "y", "z"]).astype({"marker": int, "ecke": int}).set_index(["marker", "ecke"])

    # print(project(x).reshape(-1, 2))
    return res, kameras, pictures, coords

In [ ]:
res, kameras, pictures, coords = ausgleichung(
    coords, pictures, kameras, corners)

#### Kontrolle

In [ ]:
coords.sort_index()

In [ ]:
pictures.drop(index=5, inplace=True)

In [ ]:
wolke_zeigen(coords, pictures)

In [ ]:
coords

In [83]:
def zeige_auf_bildern(coords, pictures, kameras):
    for nr in pictures.index:
        print(nr)
        imgCV = cv2.imread(pictures.query("img == "+str(nr))['pfad'].values)
        r = pictures.query("img == "+str(nr))[["r1", "r2", "r3"]].values
        t = pictures.query("img == "+str(nr))[["x", "y", "z"]].values
        kp = kameras.query("kamera == 1").values
        mtx = np.array([[kp[0][0], 0, kp[0][2]], [
            0, kp[0][1], kp[0][3]], [0, 0, 1]])
        dist = kp[0][4:]
        for p in corners.query("img == "+str(nr)).reset_index().values:
            cv2.circle(imgCV, (int(p[3]), int(p[4])), 10, (255, 0, 0), -1)
            cv2.putText(imgCV, str(int(p[1]))+'-'+str(int(p[2])), (int(p[3]),
                        int(p[4])),  cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 0, 0), 5)
        img_points, _ = cv2.projectPoints(coords.values, r, t, mtx, dist)
        for c, p in zip(coords.reset_index()['marker'].values, img_points):
            try:
                cv2.circle(imgCV, (int(p[0][0]), int(
                    p[0][1])), 10, (0, 0, 255), -1)
                cv2.putText(imgCV, str(c), (int(p[0][0]), int(
                    p[0][1])), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 2)
            except:
                pass
        
        picWC = pictures.copy()
        picWC[['rx', 'ry', 'rz']] = [-np.linalg.inv(cv2.Rodrigues(p[['r1', 'r2', 'r3']].values)[0]) @ p[['x','y','z']].values.T for i,p in pictures.iterrows()]
        cam_points, _ = cv2.projectPoints(picWC[['rx','ry','rz']].values, r, t, mtx, dist)
        for c, p in zip(picWC.reset_index()['img'].values, cam_points):
            if nr == c:
                continue
            try:
                cv2.circle(imgCV, (int(p[0][0]), int(
                    p[0][1])), 10, (0, 255, 0), -1)
                cv2.putText(imgCV, str(c+1), (int(p[0][0]), int(
                    p[0][1])), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 255, 0), 2)
            except:
                pass
        cv2.drawFrameAxes(imgCV, mtx, dist, r, t, 10,10)
        plt.figure(figsize=(14, 11))
        plt.imshow(imgCV)
        plt.show()


zeige_auf_bildern(coords, pictures, kameras)

0


TypeError: Can't convert object to 'str' for 'filename'

In [ ]:
picWC = pictures.copy()
picWC[['rx', 'ry', 'rz']] = [-np.linalg.inv(cv2.Rodrigues(np.array(list(p[['r1', 'r2', 'r3']].values)))[0]) @ p[['t1','t2','t3']].values.T for i,p in pictures.dropna().iterrows()]
picWC